# Zillow API - GetSearchResults API Functions

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Ariel Herrera arielherrera@analyticsariel.com                                                      |
| External References   | <a href="https://rapidapi.com/dimashirokov/api/Zillow/endpoints" target="_blank">(1) Rapid API - Zillow</a>, <a href="https://www.zillow.com/howto/api/GetSearchResults.htm" target="_blank">(2) Zillow  API Source Documentation</a>|
| Input Datasets        | API key, Address params                                                                                    |
| Output Datasets       | Table of getSearchResults content                                              |
| Input Data Source     | API                                                                                                |
| Output Data Source    | Dataframe                                                                                                   |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 18th Jan 2021 | Ariel Herrera | Notebook created to get search results from Zillow API. |

## Other Details
This Notebook is a prototype.

## <font color="blue">Imports</font>

In [1]:
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup

## <font color="blue">Local and Constant Variables</font>

In [2]:
pd.options.display.max_columns = None # show all columns in display

In [3]:
# read api keys file and assign variables
df = pd.read_csv('../data/input/api_keys.csv')
zwsid = df.loc[df['API'] == 'zillow']['KEY'].iloc[0]
rapid_api_key = df.loc[df['API'] == 'rapid']['KEY'].iloc[0]

In [4]:
# address API params
street = "1475 Austin Rd SW"
city = "South Fulton"
state = "GA"
zip_code = "30331"

## <font color="blue">Functions</font>

In [5]:
def zillow_get_search_results(zwsid, 
                              rapid_api_key, 
                              street, 
                              city, 
                              state, 
                              zip_code, 
                              zestimate=True,
                              rent_zestimate=True):
    """
    Finds a property for a specified address. 
    
    The content returned contains the address for the property or properties 
    as well as the Zillow Property ID (ZPID) and current Zestimate®. 
    It also includes the date the Zestimate was computed, 
    a valuation range and the Zestimate ranking for the property within its ZIP code.
    
    The GetSearchResults API Web Service is located at: 
    http://www.zillow.com/webservice/GetSearchResults

    Parameters
    ----------
    @zwsid [api_key]: The Zillow Web Service Identifier to access data from Zillow API
    @rapid_api_key [string]: Key to access data from Rapid API
    @street [string]: The street of the property to search
    @city [string]: The city of the property to search
    @state [string]: The state of the property to search
    @zip_code [string]: The zip code of the property to search
    @zestimate [boolean]: Return Rent Zestimate information if available

    Returns
    -------
    [json] API response

    """
    # adjust params for payload
    street_formatted = street.replace(" ", "%20") # URL space encoding
    city_formatted = city.replace(" ", "%20") # URL space encoding
    if zestimate == True:
        zestimate_str = "true"
    else:
        zestimate_str = "false"
    if rent_zestimate == True:
        rent_zestimate_str = "true"
    else:
        rent_zestimate_str = "false"
    
    # url for api
    url = "https://zillowdimashirokovv1.p.rapidapi.com/GetSearchResults.htm"

    # payload (data pact) to send parameters
    payload = "zws-id={0}&citystatezip={1}%2C%20{2}%20{3}&address={4}&rentzestimate={5}%2C{6}"\
                .format(zwsid, 
                        city_formatted,
                        state,
                        str(zip_code), # make sure zip code is in string format
                        street_formatted,
                        zestimate_str,
                        rent_zestimate_str)
    
    # header
    headers = {
        'content-type': "application/x-www-form-urlencoded",
        'x-rapidapi-key': rapid_api_key,
        'x-rapidapi-host': "ZillowdimashirokovV1.p.rapidapi.com"
        }
    
    # response
    response = requests.request("POST", url, data=payload, headers=headers)
    
    # check if there is an error on the request
    if response.status_code == 200: # no error
        return response.text
    else:
        return """Error {0}. See API source documentation for message and error code - https://www.zillow.com/howto/api/GetSearchResults.htm""".format(response.status_code)
    
    
def zillow_parse_search_results(response_content):
    """
    Parse search results to tabular format. 

    Parameters
    ----------
    @response_content [string]: Response text from getSearchResults API

    Returns
    -------
    [dataframe] Tabular response

    """
    soup = BeautifulSoup(get_search_results_response, "xml")
    
    content_dict = {
        "street": [soup.find('street').text],
        "city": [soup.find('city').text],
        "state": [soup.find('state').text],
        "zipcode": [soup.find('zipcode').text],
        "zpid": [soup.find('zpid').text],
        "home_details": [soup.find('homedetails').text],
        "graphsanddata": [soup.find('graphsanddata').text],
        "mapthishome": [soup.find('mapthishome').text],
        "comparables": [soup.find('comparables').text],
        "latitude": [soup.find('latitude').text],
        "longitude": [soup.find('longitude').text],
        "zestimate_amount": [soup.find('amount').text],
        "zestimate_last_updated": [soup.find('last-updated').text],
        "zestimate_value_change": [soup.find('valueChange').text],
        "zestimate_range_low": [soup.find('valuationRange').find('low').text],
        "zestimate_range_high": [soup.find('valuationRange').find('high').text]
    }

    return pd.DataFrame.from_dict(content_dict)

## <font color="blue">Data</font>

### <font color="purple">Rapid API - Zillow Request</font>

In [6]:
# get api response
get_search_results_response = zillow_get_search_results(zwsid=zwsid, 
                                                      rapid_api_key=rapid_api_key, 
                                                      street=street, 
                                                      city=city, 
                                                      state=state, 
                                                      zip_code=zip_code)

### <font color="purple">Parse Request Data</font>

In [8]:
# transform xml response to dataframe
df = zillow_parse_search_results(response_content=get_search_results_response)
df

,street,city,state,zipcode,zpid,home_details,graphsanddata,mapthishome,comparables,latitude,longitude,zestimate_amount,zestimate_last_updated,zestimate_value_change,zestimate_range_low,zestimate_range_high
0,1475 Austin Rd SW,South Fulton,GA,30331,35892041,https://www.zillow.com/homedetails/1475-Austin...,http://www.zillow.com/homedetails/1475-Austin-...,http://www.zillow.com/homes/35892041_zpid/,http://www.zillow.com/homes/comps/35892041_zpid/,33.71388,-84.510767,269918,01/14/2021,-5084,256422,283414


In [9]:
# get zestimate data
zestimate_value = df['zestimate_amount'].iloc[0]
zestimate_value

'269918'

# End Notebook